# Kryptoanalýza substituční šifry pomocí Metropolis-Hastings algoritmu

Tento notebook demonstruje implementaci a použití knihovny pro šifrování, dešifrování a kryptoanalýzu substituční šifry.

In [ ]:
# Import potřebných knihoven
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from substitution_cipher import SubstitutionCipher, BigramAnalysis
from cryptanalysis import MetropolisHastingsCryptanalysis, load_reference_matrix
import pickle
%matplotlib inline

## 1. Šifrování a dešifrování textu

Nejprve ukážeme základní funkce pro šifrování a dešifrování textu pomocí substituční šifry.

In [ ]:
# Vytvoření instance šifry
cipher = SubstitutionCipher()

# Ukázkový text
original_text = "AHOJ_JAK_SE_MAS_DNES_JE_KRASNY_DEN"
print(f"Původní text: {original_text}")

# Vygenerování náhodného klíče
key = cipher.generate_random_key()
key_string = cipher.key_to_string(key)
print(f"\nKlíč (permutace): {key_string}")
print(f"Abeceda:          {cipher.alphabet}")

# Šifrování
encrypted = cipher.encrypt(original_text, key)
print(f"\nZašifrovaný text: {encrypted}")

# Dešifrování
decrypted = cipher.decrypt(encrypted, key)
print(f"Dešifrovaný text: {decrypted}")
print(f"\nSprávně dešifrováno: {decrypted == original_text}")

## 2. Předpříprava textu

Ukázka převodu českého textu do formátu vhodného pro šifrování.

In [ ]:
# Český text s diakritikou
czech_text = "Čáp létá přes louky a zpívá si píseň."
print(f"Původní český text: {czech_text}")

# Předpříprava textu
processed = cipher.preprocess_text(czech_text)
print(f"Zpracovaný text:    {processed}")

## 3. Vytvoření bigramové matice

Vizualizace bigramové matice vytvořené z textu knihy Krakatit.

In [ ]:
# Načtení uložené bigramové matice
with open('data/czech_bigram_data.pkl', 'rb') as f:
    bigram_data = pickle.load(f)

ref_matrix = bigram_data['matrix']
alphabet = bigram_data['alphabet']
source_length = bigram_data['source_text_length']

print(f"Bigramová matice vytvořena z textu o délce {source_length:,} znaků")
print(f"Rozměry matice: {ref_matrix.shape}")

In [ ]:
# Vizualizace bigramové matice
plt.figure(figsize=(14, 12))

# Použití logaritmického měřítka pro lepší vizualizaci
log_matrix = np.log10(ref_matrix + 1e-10)

sns.heatmap(log_matrix, 
            xticklabels=list(alphabet),
            yticklabels=list(alphabet),
            cmap='viridis',
            cbar_kws={'label': 'log10(pravděpodobnost)'})

plt.title('Bigramová matice českého textu (logaritmické měřítko)', fontsize=16)
plt.xlabel('Druhý znak', fontsize=12)
plt.ylabel('První znak', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Nejčastější bigramy
analyzer = BigramAnalysis()
print("20 nejčastějších bigramů v českém textu:")
print("="*40)

# Najdi indexy největších hodnot
flat_indices = np.argsort(ref_matrix.ravel())[-20:][::-1]
indices = np.unravel_index(flat_indices, ref_matrix.shape)

for i in range(20):
    row, col = indices[0][i], indices[1][i]
    bigram = alphabet[row] + alphabet[col]
    prob = ref_matrix[row, col]
    print(f"{i+1:2d}. {bigram}: {prob:.4f} ({prob*100:.2f}%)")

## 4. Kryptoanalýza pomocí Metropolis-Hastings algoritmu

Demonstrace automatického prolomení šifry.

In [ ]:
# Vytvoření testovacího textu
test_text = """NEBYLA_A_JA_V_LENOSCE_JAKO_KUS_DREVA_VIS_UNAVEN_PRILIS_PRACE_A_NAJEDNOU_PRASK""".replace('\n', '')

print(f"Původní text ({len(test_text)} znaků):")
print(test_text[:100] + "...")

# Zašifrování
test_key = cipher.generate_random_key()
encrypted_test = cipher.encrypt(test_text, test_key)

print(f"\nZašifrovaný text:")
print(encrypted_test[:100] + "...")
print(f"\nPoužitý klíč: {cipher.key_to_string(test_key)}")

In [ ]:
# Kryptoanalýza
cryptanalysis = MetropolisHastingsCryptanalysis(ref_matrix, temperature=2.0)

print("Spouštím kryptoanalýzu...")
print("(Toto může trvat několik minut)\n")

found_key, decrypted_text, fitness_history = cryptanalysis.metropolis_hastings(
    encrypted_test, 
    iterations=20000,
    print_progress=True
)

In [ ]:
# Výsledky kryptoanalýzy
print("\nVÝSLEDKY KRYPTOANALÝZY:")
print("="*50)
print(f"\nDešifrovaný text:")
print(decrypted_text[:100] + "...")

print(f"\nNalezený klíč: {cipher.key_to_string(found_key)}")
print(f"Správný klíč:  {cipher.key_to_string(test_key)}")

# Porovnání
correct_chars = sum(1 for orig, dec in zip(test_text, decrypted_text) if orig == dec)
accuracy = correct_chars / len(test_text) * 100
print(f"\nPřesnost dešifrování: {accuracy:.2f}%")

In [ ]:
# Vizualizace průběhu optimalizace
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(fitness_history)
plt.title('Vývoj fitness během optimalizace')
plt.xlabel('Iterace')
plt.ylabel('Fitness (log-pravděpodobnost)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(fitness_history[-1000:])
plt.title('Posledních 1000 iterací')
plt.xlabel('Iterace')
plt.ylabel('Fitness')
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Analýza vlivu délky textu na úspěšnost dešifrování

Testujeme, jak délka zašifrovaného textu ovlivňuje úspěšnost kryptoanalýzy.

In [ ]:
# Test různých délek textu
lengths = [50, 100, 250, 500, 1000]
results = []

# Načtení dlouhého textu pro testování
with open('data/krakatit_processed.txt', 'r') as f:
    long_text = f.read()

for length in lengths:
    print(f"\nTestuji text o délce {length} znaků...")
    
    # Vyber část textu
    sample_text = long_text[:length]
    
    # Zašifruj
    sample_key = cipher.generate_random_key()
    encrypted_sample = cipher.encrypt(sample_text, sample_key)
    
    # Dešifruj (méně iterací pro rychlost)
    found_key, decrypted, history = cryptanalysis.metropolis_hastings(
        encrypted_sample, 
        iterations=5000,
        print_progress=False
    )
    
    # Vyhodnoť úspěšnost
    correct = sum(1 for orig, dec in zip(sample_text, decrypted) if orig == dec)
    accuracy = correct / length * 100
    
    results.append({
        'length': length,
        'accuracy': accuracy,
        'final_fitness': history[-1]
    })
    
    print(f"Přesnost: {accuracy:.2f}%")

In [ ]:
# Vizualizace výsledků
plt.figure(figsize=(10, 6))

lengths_list = [r['length'] for r in results]
accuracies = [r['accuracy'] for r in results]

plt.plot(lengths_list, accuracies, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Délka textu (znaky)', fontsize=12)
plt.ylabel('Přesnost dešifrování (%)', fontsize=12)
plt.title('Vliv délky textu na úspěšnost kryptoanalýzy', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0, 105)

# Přidání hodnot k bodům
for length, acc in zip(lengths_list, accuracies):
    plt.annotate(f'{acc:.1f}%', (length, acc), 
                textcoords="offset points", xytext=(0,10), ha='center')

plt.tight_layout()
plt.show()

## 6. Shrnutí

Implementovaná knihovna umožňuje:

1. **Šifrování a dešifrování** textu pomocí substituční šifry
2. **Vytvoření bigramové matice** z referenčního českého textu
3. **Automatickou kryptoanalýzu** pomocí Metropolis-Hastings algoritmu
4. **Vizualizaci** výsledků a průběhu optimalizace

Úspěšnost kryptoanalýzy závisí především na:
- Délce zašifrovaného textu (delší texty jsou snadněji dešifrovatelné)
- Kvalitě referenční bigramové matice
- Parametrech algoritmu (počet iterací, teplota)

Algoritmus dosahuje velmi dobrých výsledků již při textech o délce několika stovek znaků.